# 1. Load the dataset

### [KOR]  

데이터셋은 한국의 응급의료 분야에서 사용되는 KTAS (Korean Triage and Acuity Scale) 분류 데이터를 기반으로 합니다. 이 데이터셋은 보건복지부의 제 2015-243호 "한국 응급환자 중증도 분류기준"에 따라 구성되어 있으며, 총 4,368개의 분류 항목으로 이루어져 있습니다.

각 데이터 항목은 다음과 같은 정보로 구성되어 있습니다.
- KTAS1_Group: 환자의 나이를 기준으로 15세를 기준으로 성인(A)과 소아(B)로 구분됩니다. 
- KTAS2_Group & KTAS3_Group: 각각 응급 상황의 대분류와 소분류를 나타냅니다. 
- KTAS4_Group: 환자의 구체적인 상태나 상황을 상세하게 나타내는 분류입니다. 
- KTAS_Code: 각 분류 항목의 고유 코드입니다. 
- KTAS_Level: 환자의 응급 상태를 1~5의 척도로 나타내는 지표로, 1이 가장 긴급하고 5가 상대적으로 덜 긴급한 상황을 의미합니다.

이 데이터셋의 목적은 각 분류 항목의 내용을 통합하여 해당 상황이나 조건에 대한 벡터 임베딩을 생성하는 것입니다. 이렇게 생성된 벡터 임베딩은 다양한 응급의료 시스템이나 애플리케이션에서 환자의 상태나 필요한 응급 조치를 빠르게 판단하는 데 사용될 수 있습니다.

### [ENG]
The dataset is based on the KTAS (Korean Triage and Acuity Scale) classification used in the emergency medical field in Korea. This dataset is structured according to the Ministry of Health and Welfare's 2015-243 "Korean Emergency Patient Severity Classification Standards", comprising a total of 4,368 classification entries.

Each data entry is characterized by the following information:
- KTAS1_Group: Classifies patients based on age, distinguishing between adults (A) and children (B) using 15 years as the threshold.
- KTAS2_Group & KTAS3_Group: Represent the major and minor categories of emergency situations, respectively.
- KTAS4_Group: Provides a detailed classification reflecting the specific condition or situation of the patient.
- KTAS_Code: A unique code for each classification entry.
- KTAS_Level: Indicates the urgency of the patient's situation on a scale of 1 to 5, where 1 represents the most urgent cases and 5 indicates less urgent situations.  

The objective of this dataset is to create vector embeddings for each classification entry by integrating the content of each group. These generated vector embeddings can be used in various emergency medical systems or applications to swiftly assess a patient's condition and determine the required emergency measures.


In [2]:
# imports
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding

In [3]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [22]:
# load & inspect dataset
input_datapath = "./KTAS_DATA.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0, encoding='cp949')
df["Combined"] = df.KTAS2_Group.str.strip() + " - " + df.KTAS3_Group.str.strip() + " - " + df.KTAS4_Group.str.strip()

encoding = tiktoken.get_encoding(embedding_encoding)
df["n_tokens"] = df.Combined.apply(lambda x: len(encoding.encode(x)))
df.head()

,KTAS1_Group,KTAS1_Code,KTAS2_Group,KTAS2_Code,KTAS3_Group,KTAS3_Code,KTAS4_Group,KTAS4_Code,KTAS_CODE,KTAS_Level,Combined,n_tokens
Index,,,,,,,,,,,,
1,성인,A,물질오용,A,물질오용/중독,A,중증 호흡곤란,AA,AAAAA,1,물질오용 - 물질오용/중독 - 중증 호흡곤란,32
2,성인,A,물질오용,A,물질오용/중독,A,쇼크,AB,AAAAB,1,물질오용 - 물질오용/중독 - 쇼크,25
3,성인,A,물질오용,A,물질오용/중독,A,무의식(GCS 3-8),AC,AAAAC,1,물질오용 - 물질오용/중독 - 무의식(GCS 3-8),32
4,성인,A,물질오용,A,물질오용/중독,A,중등도 호흡곤란,AD,AAAAD,2,물질오용 - 물질오용/중독 - 중등도 호흡곤란,33
5,성인,A,물질오용,A,물질오용/중독,A,혈역학적 장애,AE,AAAAE,2,물질오용 - 물질오용/중독 - 혈역학적 장애,32


# 2. Get embeddings and save them for future reuse

In [ ]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage
import openai
openai.api_key = input()


# This may take a few minutes
df["Embedding"] = df.Combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("./KTAS_DATA_Embeddings.csv")